In [13]:

import sys
print(sys.executable)

c:\Users\jimix\AppData\Local\Programs\Python\Python312\python.exe


In [14]:

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from einops import rearrange
import timm
import os
from tqdm import tqdm



In [15]:
run1 = np.load("../data/Run355456_Dataset_jqkne.npy")
run2 = np.load("../data/Run357479_Dataset_iodic.npy")

labels1 = np.zeros(len(run1))
labels2 = np.ones(len(run2))

X = np.concatenate([run1, run2], axis=0)
y = np.concatenate([labels1, labels2], axis=0)

X = X / X.max()
X = X[:, np.newaxis, :, :]  # Shape: (20000, 1, 64, 72)

In [16]:
X = np.pad(X, ((0,0),(0,0),(0,0),(4,0)))  # Pad left to make (64,72) → (64,76)

In [18]:
class DQMDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = DQMDataset(X, y)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:
class SimpleViT(nn.Module):
    def __init__(self):
        super().__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=False, in_chans=1, num_classes=2)

    def forward(self, x):
        x = nn.functional.interpolate(x, size=(224, 224), mode='bilinear')
        return self.vit(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleViT().to(device)

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        print(f"Epoch {epoch+1} | Batch {batch_idx+1}/{len(train_loader)}", end='\r')

    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(train_loader):.4f}")

KeyboardInterrupt: 

In [ ]:
model.eval()
y_true, y_pred, y_prob = [], [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        probs = torch.softmax(outputs, dim=1)[:,1].cpu().numpy()
        preds = torch.argmax(outputs, dim=1).cpu().numpy()

        y_true.extend(labels.numpy())
        y_pred.extend(preds)
        y_prob.extend(probs)

acc = accuracy_score(y_true, y_pred)
auc = roc_auc_score(y_true, y_prob)
fpr, tpr, _ = roc_curve(y_true, y_prob)

print(f"Accuracy: {acc:.4f}")
print(f"AUC: {auc:.4f}")

plt.figure(figsize=(6,4))
plt.plot(fpr, tpr, label=f"AUC = {auc:.4f}")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.grid(True)
plt.show()